In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

# --- 1. Environment ---
class TicTacToeEnv:
    def __init__(self):
        self.board = np.zeros(9, dtype=np.int8)
        self.agent = 1
        self.opponent = -1
        self.reset()

    def reset(self):
        self.board[:] = 0
        self.mover = np.random.choice([1, -1])
        return self.board.copy()

    def available_actions(self):
        return np.where(self.board == 0)[0]

    def reward_done(self):
        lines = [
            (0,1,2),(3,4,5),(6,7,8),
            (0,3,6),(1,4,7),(2,5,8),
            (0,4,8),(2,4,6)
        ]
        for i,j,k in lines:
            s = self.board[i] + self.board[j] + self.board[k]
            if s == 3:  return 1.0, True
            if s == -3: return -1.0, True
        if np.all(self.board != 0):
            return 0.0, True
        return 0.0, False

    def step(self, action, opponent_policy=None):
        if self.board[action] != 0:
            return self.board.copy(), -1.0, True

        self.board[action] = self.agent
        r, d = self.reward_done()
        if d:
            return self.board.copy(), r, True

        if opponent_policy is None:
            opp_action = np.random.choice(self.available_actions())
        else:
            opp_action = opponent_policy(self.board.copy(), self.available_actions())

        self.board[opp_action] = self.opponent
        r, d = self.reward_done()

        if d and r == -1:
            return self.board.copy(), -1.0, True

        return self.board.copy(), r, d

# --- 2. Model ---
class QNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 9)
        )
        self._init()

    def _init(self):
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, nonlinearity="relu")
                nn.init.zeros_(m.bias)
        nn.init.uniform_(self.net[-1].weight, -1e-3, 1e-3)

    def forward(self, x):
        return self.net(x)



# --- 3. Helpers ---
class ReplayBuffer:
    def __init__(self, capacity=50000):
        self.buf = deque(maxlen=capacity)

    def push(self, s, a, r, ns, d, next_avail):
        self.buf.append((s, a, r, ns, d, next_avail))

    def sample(self, batch):
        samples = random.sample(self.buf, batch)
        s,a,r,ns,d,na = zip(*samples)
        return (
            torch.FloatTensor(s),
            torch.LongTensor(a).unsqueeze(1),
            torch.FloatTensor(r).unsqueeze(1),
            torch.FloatTensor(ns),
            torch.FloatTensor(d).unsqueeze(1),
            na
        )

    def __len__(self):
        return len(self.buf)


def select_action(model, state, avail, epsilon):
    if random.random() < epsilon:
        return random.choice(avail)

    with torch.no_grad():
        q = model(torch.FloatTensor(state).unsqueeze(0))[0]

    mask = torch.full_like(q, -1e9)
    mask[avail] = q[avail]
    return mask.argmax().item()

def train_step(model, target, optim, buffer, batch=128, gamma=0.95):
    if len(buffer) < batch:
        return None  # <- important for clean stats handling

    s, a, r, ns, d, next_avail = buffer.sample(batch)

    q_sa = model(s).gather(1, a)

    with torch.no_grad():
        q_next = target(ns)
        masked = torch.full_like(q_next, -1e9)
        for i, avail in enumerate(next_avail):
            masked[i, avail] = q_next[i, avail]

        q_max = masked.max(1, keepdim=True)[0]
        target_q = r + gamma * q_max * (1 - d)

    td_error = target_q - q_sa
    loss = nn.SmoothL1Loss()(q_sa, target_q)

    optim.zero_grad()
    loss.backward()
    nn.utils.clip_grad_value_(model.parameters(), 1.0)
    optim.step()

    # ---- stats returned here ----
    return {
        "loss": loss.item(),
        "td_error": td_error.abs().mean().item(),
        "q_mean": q_sa.mean().item(),
    }

def train():
    env = TicTacToeEnv()
    model = QNetwork()
    target = QNetwork()
    target.load_state_dict(model.state_dict())

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    buffer = ReplayBuffer()

    steps = 0
    epsilon = 1.0

    losses = []
    ep_lengths = []

    def update_target():
        target.load_state_dict(model.state_dict())

    for episode in range(60000):
        s = env.reset()
        ep_len = 0

        if env.mover == -1:
            env.board[random.choice(env.available_actions())] = -1
            s = env.board.copy()

        done = False
        while not done:
            avail = env.available_actions()
            epsilon = max(0.01, 1.0 - steps / 40000) if episode < 50000 else 0.005

            if episode > 30000:
                for param_group in optimizer.param_groups:
                    param_group['lr'] = 1e-4

            a = select_action(model, s, avail, epsilon)
            ns, r, done = env.step(a)
            next_avail = env.available_actions() if not done else []

            buffer.push(s, a, r, ns, done, next_avail)
            s = ns
            steps += 1
            ep_len += 1

            stats = train_step(model, target, optimizer, buffer)

            if stats is not None:
                losses.append(stats["loss"])

        ep_lengths.append(ep_len)

        # --- periodic target update
        if episode % 500 == 0:
            update_target()

        # --- stats print
        if episode % 2000 == 0 and episode > 0:
            w, d, l = evaluate_vs_random(model)

            print(
                f"[Ep {episode:6d}] "
                f"ε={epsilon:.2f} | "
                f"Win={w:.2%} Draw={d:.2%} Loss={l:.2%} | "
                f"AvgLen={np.mean(ep_lengths[-2000:]):.2f} | "
                f"Loss={np.mean(losses[-5000:]):.4f}"
            )

    return model, target, optimizer, buffer

def evaluate_vs_random(model, games=500):
    env = TicTacToeEnv()
    wins = draws = losses = 0

    for _ in range(games):
        s = env.reset()
        done = False

        if env.mover == -1:
            env.board[random.choice(env.available_actions())] = -1
            s = env.board.copy()

        while not done:
            a = select_action(model, s, env.available_actions(), epsilon=0.0)
            s, r, done = env.step(a)

        if r == 1: wins += 1
        elif r == 0: draws += 1
        else: losses += 1

    return wins / games, draws / games, losses / games

In [9]:
final_model, target, optimizer, buffer  = train()

[Ep   2000] ε=0.81 | Win=84.80% Draw=5.80% Loss=9.40% | AvgLen=3.80 | Loss=0.0554
[Ep   4000] ε=0.62 | Win=89.40% Draw=3.00% Loss=7.60% | AvgLen=3.73 | Loss=0.0696
[Ep   6000] ε=0.44 | Win=93.60% Draw=2.60% Loss=3.80% | AvgLen=3.67 | Loss=0.0643
[Ep   8000] ε=0.26 | Win=89.20% Draw=4.20% Loss=6.60% | AvgLen=3.58 | Loss=0.0598
[Ep  10000] ε=0.09 | Win=87.20% Draw=6.40% Loss=6.40% | AvgLen=3.49 | Loss=0.0569
[Ep  12000] ε=0.01 | Win=94.20% Draw=2.40% Loss=3.40% | AvgLen=3.46 | Loss=0.0525
[Ep  14000] ε=0.01 | Win=91.60% Draw=3.00% Loss=5.40% | AvgLen=3.44 | Loss=0.0475
[Ep  16000] ε=0.01 | Win=92.00% Draw=3.40% Loss=4.60% | AvgLen=3.43 | Loss=0.0430
[Ep  18000] ε=0.01 | Win=93.00% Draw=4.00% Loss=3.00% | AvgLen=3.40 | Loss=0.0369
[Ep  20000] ε=0.01 | Win=92.60% Draw=5.00% Loss=2.40% | AvgLen=3.44 | Loss=0.0297
[Ep  22000] ε=0.01 | Win=94.00% Draw=3.00% Loss=3.00% | AvgLen=3.42 | Loss=0.0245
[Ep  24000] ε=0.01 | Win=91.20% Draw=2.40% Loss=6.40% | AvgLen=3.42 | Loss=0.0206
[Ep  26000] ε=0.

In [10]:
def train_resume(model, target, optimizer, buffer, start_episode=60000, num_episodes=10000):
    """Resume training from a checkpoint.
    
    Args:
        model: The trained QNetwork
        target: The target QNetwork
        optimizer: The optimizer state (or create new)
        buffer: The replay buffer (can be empty or pre-filled)
        start_episode: Episode number to resume from
        num_episodes: How many more episodes to train
    """
    env = TicTacToeEnv()
    
    steps = 0
    epsilon = 1.0
    
    losses = []
    ep_lengths = []
    
    def update_target():
        target.load_state_dict(model.state_dict())
    
    for episode in range(start_episode, start_episode + num_episodes):
        s = env.reset()
        ep_len = 0
        
        if env.mover == -1:
            env.board[random.choice(env.available_actions())] = -1
            s = env.board.copy()
        
        done = False
        while not done:
            avail = env.available_actions()
            epsilon = max(0.01, 1.0 - steps / 40000) if episode < 50000 else 0.005
            
            if episode > 30000:
                for param_group in optimizer.param_groups:
                    param_group['lr'] = 1e-4
            if episode > 50000:
                for param_group in optimizer.param_groups:
                    param_group['lr'] = 1e-5
            
            a = select_action(model, s, avail, epsilon)
            ns, r, done = env.step(a)
            next_avail = env.available_actions() if not done else []
            
            buffer.push(s, a, r, ns, done, next_avail)
            s = ns
            steps += 1
            ep_len += 1
            
            stats = train_step(model, target, optimizer, buffer)
            
            if stats is not None:
                losses.append(stats["loss"])
        
        ep_lengths.append(ep_len)
        
        if episode % 500 == 0:
            update_target()
        
        if episode % 2000 == 0 and episode > 0:
            w, d, l = evaluate_vs_random(model)
            
            print(
                f"[Ep {episode:6d}] "
                f"ε={epsilon:.2f} | "
                f"Win={w:.2%} Draw={d:.2%} Loss={l:.2%} | "
                f"AvgLen={np.mean(ep_lengths[-2000:]):.2f} | "
                f"Loss={np.mean(losses[-5000:]):.4f}"
            )
    
    return model

In [13]:
final_model = train_resume(final_model, target, optimizer, buffer, start_episode=70000, num_episodes=10000)

[Ep  70000] ε=0.01 | Win=97.20% Draw=2.20% Loss=0.60% | AvgLen=4.00 | Loss=0.0060
[Ep  72000] ε=0.01 | Win=96.80% Draw=3.20% Loss=0.00% | AvgLen=3.25 | Loss=0.0088
[Ep  74000] ε=0.01 | Win=95.60% Draw=4.20% Loss=0.20% | AvgLen=3.20 | Loss=0.0084
[Ep  76000] ε=0.01 | Win=94.60% Draw=3.40% Loss=2.00% | AvgLen=3.27 | Loss=0.0082
[Ep  78000] ε=0.01 | Win=95.40% Draw=3.20% Loss=1.40% | AvgLen=3.32 | Loss=0.0086


In [14]:
# Save the model
torch.save(final_model.state_dict(), 'model_v8.pth')
print("\nModel saved as 'model_v8.pth'")


Model saved as 'model_v8.pth'
